In [27]:
from paddleocr import PaddleOCR, draw_ocr

# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr_model = PaddleOCR(
    use_angle_cls=True, lang="en", use_gpu=False
)  # need to run only once to download and load model into memory
file_path = "../Fuel_Invoices/Cepsa.pdf"
result = ocr_model.ocr(file_path, cls=True)
# for idx in range(len(result)):
#     res = result[idx]
# for line in res:
#         print('////////////////////////////////////////////////')
#         print(line)

#     print('----------')
#     print(res)
# print('================================================')
# result

[2023/07/21 10:08:57] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Payoff/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Payoff/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, 

Deprecation: 'getPixmap' removed from class 'Page' after v1.19.0 - use 'get_pixmap'.


[2023/07/21 10:08:59] ppocr DEBUG: dt_boxes num : 188, elapse : 0.6682145595550537
[2023/07/21 10:09:01] ppocr DEBUG: cls num  : 188, elapse : 2.157578706741333
[2023/07/21 10:09:24] ppocr DEBUG: rec_res num  : 188, elapse : 22.762155771255493
[2023/07/21 10:09:25] ppocr DEBUG: dt_boxes num : 86, elapse : 0.6176130771636963
[2023/07/21 10:09:26] ppocr DEBUG: cls num  : 86, elapse : 0.9965651035308838
[2023/07/21 10:09:41] ppocr DEBUG: rec_res num  : 86, elapse : 14.862959384918213


In [28]:
# for line in result[0]:
#     print(line[1][0])
#     print('--------------------------------')


allTheTexts = [line[1][0] for line in result[0]]
allTheTexts

['Data : 22/05/2023',
 'Paseo De La Castellana, 259 A',
 'CEPSA',
 '28046 Madrid (SPAIN)',
 'V.A.T. reg.no.A28003119',
 'Reg. Merc. Madrid 206/100/Hoja 6045 Sociedades',
 'DATE :',
 'No',
 'PAG.',
 'INVOICE',
 '15.05.2023',
 '8200151210',
 '1 / 2',
 'PLACE OF SUPPLY :',
 'VAT reg.nb. :',
 'TENERIFE SUR',
 'IT08254440012',
 'CONSIGNEE :',
 'NEOS AIRWAYS',
 'NEOS AIRWAYS',
 'VIA DELLA CHIESA 68',
 '21019 SOMMA LOMBARDO (VA)',
 'TERMS OF DELIVERY :',
 'ITALY',
 'IPL - Into Plane',
 'DUE :',
 '30.05.2023',
 'NEW TERMS OF_PAYMENT.UPDATE YOUR RECORDS ACCORDINGLY:',
 'BENEFICIARY: CEPSA_TREASURY S.A.U.',
 'INVOICE RECEIVER:',
 "BENEFICIARY'BANK: BANCO BILBAO'VIZCAYA ARGENTARIA (BBVA)",
 'NEOS AIRWAYS',
 'SWIFT: BBVAESMM',
 'IBAN:ES78 0182 3994 0120 1316 0033',
 'CORRESPONDENT BANK AND SWIFT: CITIBANK N.Y.(CITIUS33)',
 'KIND CURRENCY:',
 'UNITED STATES DOLLAR (USD)',
 'CSO.FEE',
 'cso',
 'A.TAX',
 'A.TAX',
 'DELIVER.DT',
 'RECEIPT N',
 'REGIST.',
 'FLIGHT N. FRM',
 'TO',
 'LTS.',
 'US.GALLONS 

In [29]:
import csv

with open(f"{file_path[17:-4]}_ocr_result.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["Text", "Confidence"])

    writer.writerow(allTheTexts)

In [30]:
import json

json_result = []
for line in result:
    json_result.append(line)

    # json_result.append({
    #     'text': line[1][0],
    #     'confidence': line[1][1]
    # })

with open(f"{file_path[17:-4]}_ocr_result.json", "w") as f:
    json.dump(json_result, f)

In [71]:
# draw result
import fitz
from PIL import Image
import cv2
import numpy as np

imgs = []
with fitz.open(file_path) as pdf:
    for pg in range(0, pdf.pageCount):
        page = pdf[pg]
        mat = fitz.Matrix(2, 2)
        pm = page.get_pixmap(matrix=mat, alpha=False)
        # if width or height > 2000 pixels, don't enlarge the image
        if pm.width > 2000 or pm.height > 2000:
            pm = page.get_pixmap(matrix=fitz.Matrix(1, 1), alpha=False)

        img = Image.frombytes("RGB", [pm.width, pm.height], pm.samples)
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        imgs.append(img)

for idx in range(len(result)):
    res = result[idx]
    image = imgs[idx]

    boxes = [line[0] for line in res]
    texts = [line[1][0] for line in res]
    scores = [line[1][1] for line in res]

    # Draw OCR results on image
    result_with_boxes = draw_ocr(image, boxes)
    result_with_text = draw_ocr(
        image, boxes, texts, scores, font_path="./fonts/latin.ttf"
    )

    # Convert PIL image to OpenCV format
    result_with_boxes = cv2.cvtColor(np.array(result_with_boxes), cv2.COLOR_RGB2BGR)

    # Display image
    cv2.imshow("result_with_boxes", result_with_boxes)
    cv2.imshow("result_with_text", result_with_text)
    cv2.waitKey(0)

    # Save image
    cv2.imwrite(f"{file_path[17:-4]}_{idx}_box.jpg", result_with_boxes)
    cv2.imwrite(f"{file_path[17:-4]}_{idx}_text.jpg", result_with_text)

cv2.destroyAllWindows()